> _First time use of the `notebooks`: follow instructions in the `README.md` file in this directory._

**[PT]** Português

--- 

**[EN]** English



# Conversão do "Ficheiro de alunos"



---

# Convertion of the "Ficheiro de alunos"

This notebook processes a file exported from the "Ficheiro de alunos" in CSV format (command separated values), 
converting it in `Kleio` source files or importing directly into a `Timelink` database.

* For detailed information on the contents of the CSV file and the processing methodolgy see [README_ucalumni.md](README_ucalumni.md)
* For information on the `Kleio` format see [README_kleio.md](README_kleio.md).
* For the structure of the `Timelink` database see [README_timelink_db.md](README_timelink_db.md)


## Converter a partir do terminal

Usar o utilitário `import-alumni.py`, a partir da directoria raiz do repositório `fauc1537-1919`.

---

## Converting from a terminal window

Use the utility `import-alumni.py`, from whithin the repository top level directory `fauc1537-1919`.



```console

    python notebooks/import-alumni.py --help 

Exemplo de conversão a partir de uma exportação em `database/auc/PT-AUC-ELU-UC-AUC-B-001-001.CSV`

---

Example of converting from an export file at  `database/auc/PT-AUC-ELU-UC-AUC-B-001-001.CSV`

```console

    python notebooks/import-alumni.py database/auc/2023-12-28-PT-AUC-ELU-UC-AUC-B-001-001.csv \
        database/sqlite3\
        --db-connection "sqlite:///database/sqlite3/fauc.db?check_same_thread=False"\
        --rows 0 \
        --batch 500 \
        --no-dryrun

## Converter a partir deste bloco de notas

Conversão neste bloco de notas na célula seguinte.

Não fechar  o bloco de notas durante o processo, que pode demorar várias horas.

---

## Converting from this notebook

Convert from within this notebook in the next cell.

Do not close the notebook during the conversion, which might take several hours.



In [ ]:
import os
from ucalumni.importer import import_auc_alumni
from ucalumni.aluno import Aluno
from ucalumni import extractors
from ucalumni import mapping
from ucalumni.config import auc_export

cwd = os.getcwd()
print("Current dir (should end in 'notebooks'):",cwd)
catalog = f'../{auc_export}' # source directory for kleio files
destdir = '../database/sqlite3' # destination directory for database file (sqlite only)
                         # connection string for direct import to database
con_string = f"sqlite:///{destdir}/fauc.db?check_same_thread=False"
# This will store the connection string for other notebooks
# To retrieve the in another notebook use %store -r con_string
%store con_string
rows= 0                 # stop after rows, 0 to process everything
batch = 100            # number of alumni per kleio file
dryrun = False           # if true do not write, just output in notebook
echo   = False           # if true each row in csv file will be printed in notebook

import_auc_alumni(  catalog,
                    destdir,
                    con_string,
                    rows,
                    batch,
                    dryrun,
                    echo
                    )

Current dir (should end in 'notebooks'): /Users/jrc/develop/fauc1537-1919/notebooks
Stored 'con_string' (str)


/Users/jrc/develop/fauc1537-1919/.venv/lib/python3.11/site-packages/IPython/extensions/storemagic.py:229: UserWarning: This is now an optional IPython functionality, setting autorestore/con_string requires you to install the `pickleshare` library.
  db[ 'autorestore/' + arg ] = obj


FileNotFoundError: [Errno 2] No such file or directory: '../{auc_export}'

## Verificar o primeiro e último estudante

---

## Check first and last student



In [2]:
from sqlalchemy import select,desc
from sqlalchemy.orm import sessionmaker
from timelink.mhk.models import base
from timelink.mhk.models.person import Person
from timelink.mhk.models.db import TimelinkDB, SQLALCHEMY_ECHO


Session = sessionmaker()

connection_string="sqlite:///../database/sqlite3/fauc.db?check_same_thread=False"

SQLALCHEMY_ECHO = True
db = TimelinkDB(connection_string)

Session.configure(bind=db.engine())

stmt = select(Person).order_by(Person.updated).limit(1)
with Session() as session:
    first = session.execute(stmt).scalar()
    print(first.id,first.name,first.updated)

stmt = select(Person).order_by(desc(Person.updated)).limit(1)
with Session() as session:
    last = session.execute(stmt).scalar()
    print(last.id,last.name, last.updated)


140337 António Pinto Abadeço 2022-06-10 01:35:55.802017
196220 Vasco de Oliveira Zuzarte 2022-06-10 07:51:46.108991


/tmp/ipykernel_3870542/181597555.py:15: DeprecationWarning: This method is deprecated, use get_engine()
  Session.configure(bind=db.engine())


## Gerar ficheiros Kleio para posterior importação em Timelink/MHK

Esta modalidade produz ficheiros na notação `Kleio`.  

Embora exija um posterior passo de tradução e conversão, tem a vantagem de produzir 
uma representação dos dados num formato legível, de fácil permuta e conservação. 

Executar a célula seguinte converte o ficheiro de alunos para ficheiros `Kleio`, com 500 alunos em cada ficheiro.

Os ficheiros têm de ser posteriormente traduzidos e importados em Timelink/MHK.

---

## Generate Kleio for import to Timelink/MHK

This variant produces files in `Kleio` notation.

Although it requires extras steps for translation and import, it has the
advantage of producing an interpretation of the catalog in a readable format
easy to exchange and preserve.

Executing the next cell with process the FA export in to `Kleio` files, 500 students per file.

The files must be translated and imported the Timelink/MHK at a later stage.

In [1]:
from ucalumni.importer import import_auc_alumni
from ucalumni.aluno import Aluno
from ucalumni import extractors
from ucalumni import mapping
from ucalumni.config import auc_export

catalog = f'../{auc_export}'
destdir = '../sources/' # destination directory for Kleio files
con_string = None         # connection string for direct import to database
rows= 10                 # stop after rows, 0 to process everything
nalumni = 3            # limit of alumni per kleio file
dryrun = True           # if true do not write, just output in notebook
echo   = False           # if true each row in csv file will be printed in notebook
import_auc_alumni(catalog,
                    destdir,
                    con_string,
                    rows,
                    nalumni,
                    dryrun,
                    echo
                    )

-- Number of records per batch reached, new batch generated. 3 lista-264605-00001 António José Abelha
Testing. would write to ../sources/A/lista-A-00001.cli
kleio$gacto2.str/obs="Generated from Archeevo export file:  ../database/auc/2023-12-28-PT-AUC-ELU-UC-AUC-B-001-001.csv dated 2023-12-28 11:45"
 fonte$lista-A-00001/data=1537-00-00:1919-11-14/loc=Arquivo da Universidade de Coimbra/ref="PT/AUC/ELU/UC-AUC/B/001-001"/tipo=UC-AUC-ficheiro-alunos/obs="""      Âmbito e conteúdo

       O índice de alunos da Universidade de Coimbra, também conhecido, apenas, como ficheiro de alunos, foi elaborado entre 1940 e 1950, pelos funcionários do Arquivo, por decisão do doutor Mário Brandão, então diretor do Arquivo da Universidade, como o atestam os relatórios de atividades enviados anualmente ao reitor da Universidade (v. Correspondência expedida do Arquivo (SR), cx.2 (1942-1947) e cx. 3 (1948-1951) – cota AUC-IV-2.ªE-10-2-18 e 19).
       Os citados relatórios revelam, por exemplo, que em 1941-19

TypeError: unbound method type.mro() needs an argument